# Plant and author distributions
Code to generate figure demonstrating the relationship between desiccation plant distribution and author locations.

In [1]:
import pandas as pd

## Read in data

In [2]:
attendees_2016 = pd.read_csv('../data/conference_data/attendees_2016.csv')
attendees_2024 = pd.read_csv('../data/conference_data/attendees_2024.csv')

In [14]:
spec_occ = pd.read_csv('../data/conference_data/species_occ.csv', encoding_errors='replace')
spec_occ.head()

,sp_id,institutionCode,catalogNumber,countryCode,decimalLongitude,decimalLatitude,source
0,aaa023,PERTH,PERTH 3249093,AUS,116.083333,-34.766667,https://www.gbif.org/
1,aaa023,PERTH,PERTH 5453119,AUS,117.270833,-34.851389,https://www.gbif.org/
2,aaa023,PERTH,PERTH 6905811,AUS,117.016667,-34.516667,https://www.gbif.org/
3,aaa023,PERTH,PERTH 5900085,AUS,115.083333,-33.750000,https://www.gbif.org/
4,aaa023,PERTH,PERTH 3248895,AUS,115.938056,-34.508056,https://www.gbif.org/


In [16]:
spec_names = pd.read_csv('../data/conference_data/species_list.csv', encoding_errors='replace')
spec_names.head()

,sp_id,Family,Species,Taxonomic authority,Taxonomic observations,Cited as DT by
0,aaf003,Anemiaceae,Anemia ferruginea,Kunth,NaN,Ibisch et al. (1995)
1,aaf019,Anemiaceae,Anemia flexuosa,(Savigny) Sw.,NaN,Meirelles et al. (1997)
2,aax231,Anemiaceae,Anemia mexicana,Klotzsch,NaN,"Kessler & Siorak (2007), Mkhize (2018)"
3,aar329,Anemiaceae,Anemia rotundifolia,Schrad.,NaN,"Kessler & Siorak (2007), Derzhavina (2017), Mk..."
4,aat020,Anemiaceae,Anemia tomentosa,(Savigny) Sw.,NaN,"Gaff (1987), Kessler & Siorak (2007), Hietz (2..."
